# Data Collection and Cleaning

1. Use lyricsgenius and get all lyrics
  - Exclude remixes, live versions, non-Taylor's version versions
  - stored in ```Lyrics_TaylorSwift.json```
2. Clean title and lyrics (remove unicode characters, empty spaces, junk text)
3. Generate json file with title, lyrics

## Scraping lyrics

In [2]:
import requests
import json
import time
import lyricsgenius as lg


In [4]:
# Set up API access
GENIUS_API_TOKEN = "YIHKEmRaW8tuw7qpsPhPf3_BZCEIp9N53yr2BsNOWi_qvjCA-WVskFIK7hL-tW6Q"
BASE_URL = "https://api.genius.com"
headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}

# Define artist to search for
artist_name = "Taylor Swift"

In [15]:
genius = lg.Genius(GENIUS_API_TOKEN, skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)

In [ ]:
genius.timeout = 8
# genius.sleep_time = 40
try:
  artist = genius.search_artist(artist_name)



In [20]:
from requests.exceptions import HTTPError, Timeout
genius.timeout = 8

try:
    artist = genius.search_artist(artist_name)
except HTTPError as e:
    print(e.errno)    # status code
    print(e.args[0])  # status code
    print(e.args[1])  # error message
except Timeout:
    pass


Searching for songs by Taylor Swift...

Song 1: "All Too Well (10 Minute Version) (Taylor’s Version) [From the Vault]"
"All Too Well (10 Minute Version) (Taylor’s Version) [Live Acoustic]" is not valid. Skipping.
Song 2: "​cardigan"
Song 3: "​exile"
Song 4: "Lover"
Song 5: "​the 1"
Song 6: "​august"
Song 7: "Look What You Made Me Do"
Song 8: "​betty"
Song 9: "​champagne problems"
Song 10: "Blank Space"
Song 11: "End Game"
Song 12: "Anti-Hero"
Song 13: "​my tears ricochet"
Song 14: "You Need To Calm Down"
Song 15: "ME!"
Song 16: "​willow"
Song 17: "...Ready for It?"
Song 18: "Cruel Summer"
Song 19: "​invisible string"
Song 20: "Style"
Song 21: "Maroon"
Song 22: "All Too Well"
Song 23: "​the last great american dynasty"
Song 24: "​seven"
Song 25: "​this is me trying"
Song 26: "​happiness"
Song 27: "You’re On Your Own, Kid"
Song 28: "Call It What You Want"
Song 29: "Delicate"
Song 30: "​illicit affairs"
Song 31: "​tolerate it"
Song 32: "Love Story"
Song 33: "Question...?"
Song 34: "Snow O

In [21]:
artist.save_lyrics()

Wrote Lyrics_TaylorSwift.json.


## Cleaning song title and lyrics

In [133]:
from string import printable
import re

## Function to take a string and clean it of non-printable characters
def clean_str (input_str):
  # tmp1 = ''.join(c for c in input_str if c.isprintable())
  output_str = re.sub(r'[^\x00-\x7f]', r'', input_str)
  # tmp2 = tmp1.encode("ascii", "ignore")
  # output_str = tmp1.decode()

  return output_str.strip()


In [134]:
unique_titles = []

def clean_song (song):
  title=clean_str(song.title)

  #Check if the song is a re-recorded version/Remove non-song entries (speeches, dates, guide, costumes)
  
  if (title in unique_titles):
    print("Repeated title")
    return {}

  if ("[Discography List]" in title or
      "Magazine" in title or
      "[Foreword]" in title or
      "[Prologue]" in title or
      "Speech" in title or
      "Costumes" in title or
      "Dates" in title or
      "Guide" in title or
      "Introduction" in title or
      "Outfit" in title or
      "*" in title or
      "Album" in title):
    print("Non-song")
    return {}

  if(title.find("(")!=-1):
    idx = title.find("(")
    root_title = title[:idx].strip()

    if (root_title in unique_titles):
      print("Repeated title")
      return {}
    else:
      title=root_title
    
  try:
    junk, l = song.lyrics.split('\n', 1)
    # print("Junk Removed")

  except:
    print("No junk")
    l=song.lyrics
    
  finally:
    lyrics=clean_str(l)

    d= {'title': title,
        'lyrics': lyrics,
        'keywords':[]}

    unique_titles.append(title)

    return d


In [135]:
#Number of valid songs
song_ct=0

# Dictionary to store contents of json file
data = {}
data['songs'] = []


for i, song in enumerate(artist.songs):
  # print(song.title)
  print("Song #", i)
  d = clean_song(song)
  if (d=={}):
    continue
  data['songs'].append(d)
  song_ct+=1



Song # 0
Song # 1
Song # 2
Song # 3
Song # 4
Song # 5
Song # 6
Song # 7
Song # 8
Song # 9
Song # 10
Song # 11
Song # 12
Song # 13
Song # 14
Song # 15
Song # 16
Song # 17
Song # 18
Song # 19
Song # 20
Song # 21
Repeated title
Song # 22
Song # 23
Song # 24
Song # 25
Song # 26
Song # 27
Song # 28
Song # 29
Song # 30
Song # 31
Song # 32
Song # 33
Song # 34
Song # 35
Song # 36
Song # 37
Song # 38
Song # 39
Song # 40
Song # 41
Song # 42
Song # 43
Song # 44
Song # 45
Song # 46
Song # 47
Song # 48
Song # 49
Song # 50
Song # 51
Song # 52
Song # 53
Song # 54
Song # 55
Song # 56
Song # 57
Song # 58
Song # 59
Song # 60
Song # 61
Song # 62
Song # 63
Song # 64
Song # 65
Song # 66
Song # 67
Song # 68
Song # 69
Song # 70
Song # 71
Song # 72
Song # 73
Song # 74
Song # 75
Song # 76
Song # 77
Song # 78
Song # 79
Song # 80
Song # 81
Song # 82
Song # 83
Song # 84
Song # 85
Song # 86
Song # 87
Song # 88
Song # 89
Song # 90
Song # 91
Song # 92
Song # 93
Song # 94
Song # 95
Song # 96
Song # 97
Song # 98
Song 

In [136]:
song_ct

373

In [137]:
# Writing into a json file

import json

json_object = json.dumps(data, indent=2)

# Writing to sample.json
with open("ts_data.json", "w") as outfile:
    outfile.write(json_object)


In [98]:
print(data['songs'][1]['lyrics'])

Vintage tee, brand new phoneHigh heels on cobblestonesWhen you are young, they assume you knownothingSequinsmile, black lipstickSensualpoliticsWhen you are young, they assumeyou know nothingBut I knew youDancin' in your Levi'sDrunk under a streetlight, II knew youHand under my sweatshirtBaby, kiss it better, IAnd when I felt like I was an old cardiganUnder someone's bedYou put me on and said I was your favoriteA friend to all is a friend to noneChase two girls, lose the oneWhen you are young, they assume you know nothingSee Taylor Swift LiveGet tickets as low as $650You might also likeBut I knew youPlaying hide-and-seek andGiving me your weekends, II knew youYour heartbeat on the High LineOnce in twenty lifetimes, IAnd when I felt like I was an old cardiganUnder someone's bedYou put me on and said I was your favoriteTo kiss in cars and downtown barsWas all we neededYou drew stars around my scarsBut now I'm bleedin''Cause I knew youSteppin' on the last trainMarked me like a bloodstain, 

In [116]:
"Date" in "Eras Tour Dat"

False